In [5]:
import torch
import torch.nn as nn
import os
import torch.optim as optim
import torchvision
import torch.nn.functional as F
from PIL import Image
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
import matplotlib.pyplot as plt

In [14]:
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),  # Convert images to grayscale
    transforms.Resize((200, 50)),  # Resize to a fixed size
    transforms.ToTensor(),          # Convert image to PyTorch tensor
    transforms.Normalize([0.5], [0.5])  # Normalize with ImageNet stats
])

# Custom dataset class
class CustomImageDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir) if fname.endswith(('.png', '.jpg', '.jpeg'))]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        # Extract the label from the filename (without extension)
        label = os.path.splitext(os.path.basename(img_path))[0]
        # Convert label to a tensor (e.g., using a simple encoding scheme)
        label_tensor = torch.tensor([ord(char) for char in label], dtype=torch.long)
        return image, label

# Create the dataset
dataset = CustomImageDataset(root_dir=os.path.expanduser('~/Downloads/captcha'), transform=transform)
if len(dataset) == 0:
    print("Dataset is empty")
# Print out the labels (base names) in the dataset
for idx, (image, label) in enumerate(dataset):
    print(f"Index: {idx}, Label: {label}")

# Create DataLoader objects
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)


# Example of iterating through the DataLoader
for batch_idx, (images, labels) in enumerate(dataloader):
    print(f"Batch {batch_idx}, Labels: {labels}")

Index: 0, Label: p5g5m
Index: 1, Label: e72cd
Index: 2, Label: pgmn2
Index: 3, Label: 6cm6m
Index: 4, Label: 68wfd
Index: 5, Label: byc82
Index: 6, Label: gcx6f
Index: 7, Label: d8dce
Index: 8, Label: gd4mf
Index: 9, Label: fy2nd
Index: 10, Label: 37ep6
Index: 11, Label: 4nnf3
Index: 12, Label: ygenn
Index: 13, Label: c4527
Index: 14, Label: x37bf
Index: 15, Label: mx8bb
Index: 16, Label: ncfgb
Index: 17, Label: ewnx8
Index: 18, Label: 2nx38
Index: 19, Label: wmpmp
Index: 20, Label: pyf65
Index: 21, Label: nxxf8
Index: 22, Label: ny5dp
Index: 23, Label: dgcm4
Index: 24, Label: cfp86
Index: 25, Label: 2xc2n
Index: 26, Label: 378e5
Index: 27, Label: d22y5
Index: 28, Label: w4nfx
Index: 29, Label: 7mgmf
Index: 30, Label: 25p2m
Index: 31, Label: nbwnn
Index: 32, Label: egxmp
Index: 33, Label: 56ncx
Index: 34, Label: gnbn4
Index: 35, Label: gw468
Index: 36, Label: n8ydd
Index: 37, Label: n7meb
Index: 38, Label: 36w25
Index: 39, Label: nnf8b
Index: 40, Label: cewnm
Index: 41, Label: g2fnw
In

In [6]:
# Step 3: Define the split ratio
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

# Step 4: Use random_split to split the dataset
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Step 5: Create DataLoader objects
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
for X, y in test_loader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break
print(f"Number of samples in train_loader: {len(train_loader.dataset)}")
print(f"Number of samples in train_loader: {len(test_loader.dataset)}")

Shape of X [N, C, H, W]: torch.Size([64, 1, 200, 50])
Shape of y: torch.Size([64]) torch.int64
Number of samples in train_loader: 856
Number of samples in train_loader: 214


In [7]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(200*50, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 36) # 5 characters by 62 possible values
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=10000, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=36, bias=True)
  )
)


In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        #y = y.view(-1)

        # Compute prediction error
        pred = model(X)
        #pred = pred.view(-1, 62)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            #y = y.view(-1)

            pred = model(X)
            #pred = pred.view(-1, 62)  # Reshape to (batch_size * 5, 62)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    #return test_loss, correct * 100


: 

In [11]:
model = NeuralNetwork().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    test(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


IndexError: Target 120 is out of bounds.